In [ ]:
from google.colab import drive
import os
import random
import numpy as np
import torch
from torchvision import transforms
from PIL import Image, ImageEnhance
from scipy.ndimage import gaussian_filter
from skimage.util import random_noise
from skimage import exposure


drive.mount('/content/drive')


def mixup_data(x, y, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    mixed_x = lam * x + (1 - lam) * y
    return mixed_x


def random_enhance(image):
    enhance_methods = [
        "brightness", "contrast", "saturation", "color_transform",
        "gaussian_noise", "gaussian_blur", "elastic_transform", 
        "color_mapping", "invert", "elastic_transform"
    ]
    
    method = random.choice(enhance_methods)
    
    if method == "brightness":
        enhancer = ImageEnhance.Brightness(image)
        return enhancer.enhance(random.uniform(0.5, 1.5))
    elif method == "contrast":
        enhancer = ImageEnhance.Contrast(image)
        return enhancer.enhance(random.uniform(0.5, 1.5))
    elif method == "saturation":
        enhancer = ImageEnhance.Color(image)
        return enhancer.enhance(random.uniform(0.5, 1.5))
    elif method == "color_transform":
        image = np.array(image)
        image = exposure.adjust_gamma(image, gamma=random.uniform(0.5, 1.5))
        return Image.fromarray(np.uint8(image))
    elif method == "gaussian_noise":
        image = np.array(image) / 255.0
        noisy_image = random_noise(image, mode='gaussian', var=0.01)
        return Image.fromarray((noisy_image * 255).astype(np.uint8))
    elif method == "gaussian_blur":
        image = np.array(image)
        blurred_image = gaussian_filter(image, sigma=random.uniform(0.5, 1.5))
        return Image.fromarray(np.uint8(blurred_image))
    elif method == "elastic_transform":
        
        image = image.transform(image.size, Image.QUAD, data=(1, 0, 0, 1, 0, 0, 1, 1))
        return image
    elif method == "color_mapping":
        image = np.array(image)
        image = exposure.equalize_hist(image) * 255
        return Image.fromarray(np.uint8(image))
    elif method == "invert":
        image = Image.fromarray(255 - np.array(image))
        return image
    else:
        return image


def process_images(dataset_path):
    categories = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]
    
    for category in categories:
        category_path = os.path.join(dataset_path, category)
        mixup_path = os.path.join(category_path, "MixUp")
        
        
        if not os.path.exists(mixup_path):
            os.makedirs(mixup_path)
        
       
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            
            if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            
          
            original_image = Image.open(img_path).convert("RGB")
            
          
            enhanced_image = random_enhance(original_image)
            
          
            transform = transforms.ToTensor()
            original_tensor = transform(original_image)
            enhanced_tensor = transform(enhanced_image)
            
        
            mixed_tensor = mixup_data(original_tensor, enhanced_tensor)
            
         
            mixed_image = transforms.ToPILImage()(mixed_tensor)
            
          
            mixup_img_name = f"mixup_{img_name}"
            mixed_image.save(os.path.join(mixup_path, mixup_img_name))
            
            print(f"Processed {img_name} -> {mixup_img_name}")


if __name__ == "__main__":
    dataset_path = "/content/drive/My Drive/MRI/Training/"
    process_images(dataset_path)
